In [1]:
!pip install -q fastapi uvicorn nest_asyncio pyngrok joblib pandas numpy

import nest_asyncio, joblib, pandas as pd, numpy as np
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
from google.colab import files

uploaded = files.upload()   # will open a file picker


Saving rf_ton_iot_pipeline.joblib to rf_ton_iot_pipeline.joblib


In [3]:
from google.colab import files

uploaded = files.upload()   # will open a file picker

Saving xgb_ton_iot_pipeline.joblib to xgb_ton_iot_pipeline.joblib


In [4]:
import joblib

rf_model = joblib.load("rf_ton_iot_pipeline.joblib")
xgb_model = joblib.load("xgb_ton_iot_pipeline.joblib")

print("Loaded both pipelines.")


Loaded both pipelines.


/usr/lib/python3.12/pickle.py:1760: UserWarning: [10:47:38] WARNING: /workspace/src/collective/../data/../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [5]:
from fastapi import FastAPI, Query
from pydantic import BaseModel
import pandas as pd
import numpy as np

app = FastAPI(title="MedSecureAI IDS", version="1.1")

class TrafficEvent(BaseModel):
    payload: dict

def required_cols(model):
    """
    Get the raw feature columns the pipeline was trained on.
    Works with sklearn >=1.0 ColumnTransformer.
    """
    try:
        return list(model.named_steps["pre"].feature_names_in_)
    except Exception:
        # Fallback: if not available, keep empty (no alignment)
        return []

# Precompute the required columns for each model
RF_COLS  = required_cols(rf_model)
XGB_COLS = required_cols(xgb_model)

def align_to_schema(payload: dict, cols: list[str]) -> pd.DataFrame:
    """Create a 1-row DataFrame with exactly the expected columns."""
    X = pd.DataFrame([payload])
    if cols:
        # add any missing columns as NaN; drop unexpected extras
        X = X.reindex(columns=cols, fill_value=np.nan)
    return X

@app.get("/health")
def health():
    return {
        "status": "ok",
        "models": ["rf", "xgb"],
        "rf_expected_cols": len(RF_COLS),
        "xgb_expected_cols": len(XGB_COLS)
    }

@app.post("/predict")
def predict(evt: TrafficEvent, model: str = Query("rf", enum=["rf", "xgb"])):
    try:
        chosen_model = rf_model if model == "rf" else xgb_model
        cols = RF_COLS if model == "rf" else XGB_COLS

        X = align_to_schema(evt.payload, cols)

        # predict proba if supported
        prob = float(chosen_model.predict_proba(X)[:, 1][0]) \
               if hasattr(chosen_model, "predict_proba") else None
        pred = int(chosen_model.predict(X)[0])

        return {
            "model": model,
            "prediction": pred,
            "probability": prob,
            "missing_columns_filled": [c for c in cols if c not in evt.payload],
            "received_keys": list(evt.payload.keys())
        }
    except Exception as e:
        # return the actual error to help debugging
        return {"error": str(e), "payload_received": evt.payload}


In [6]:
!pip install -q pyngrok
from pyngrok import ngrok

ngrok.set_auth_token("32NJXIDUDoqLgQWeeRkM9pxZgB1_VdVWX8zAz9ip7U45k9VK")


In [7]:
import nest_asyncio, uvicorn
from pyngrok import ngrok

nest_asyncio.apply()
public_url = ngrok.connect(8000, "http")
print("Public URL:", public_url)
uvicorn.run(app, host="0.0.0.0", port=8000)




ERROR:pyngrok.process.ngrok:t=2025-09-21T10:47:46+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-21T10:47:46+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

In [ ]:
!pip install -q shap


In [ ]:
from fastapi import FastAPI, Query
from pydantic import BaseModel
import pandas as pd, numpy as np, json, time, shap
from sklearn.utils.validation import check_is_fitted

# sanity: ensure models are fitted
check_is_fitted(rf_model)
check_is_fitted(xgb_model)

app = FastAPI(title="MedSecureAI IDS", version="2.0")

class TrafficEvent(BaseModel):
    payload: dict

LOG_PATH = "predictions_log.jsonl"

def log_prediction(record: dict):
    with open(LOG_PATH, "a") as f:
        f.write(json.dumps(record) + "\n")

def required_cols(model):
    # raw feature columns the ColumnTransformer expects
    try:
        return list(model.named_steps["pre"].feature_names_in_)
    except Exception:
        return []

RF_COLS  = required_cols(rf_model)
XGB_COLS = required_cols(xgb_model)

def align_to_schema(payload: dict, cols: list[str]) -> pd.DataFrame:
    X = pd.DataFrame([payload])
    return X.reindex(columns=cols, fill_value=np.nan) if cols else X

@app.get("/health")
def health():
    return {"status": "ok", "models": ["rf", "xgb"]}

@app.get("/schema")
def schema(model: str = Query("rf", enum=["rf", "xgb"])):
    cols = RF_COLS if model == "rf" else XGB_COLS
    return {"model": model, "expected_columns": cols, "count": len(cols)}

@app.post("/predict")
def predict(evt: TrafficEvent, model: str = Query("rf", enum=["rf", "xgb"])):
    ts = time.time()
    try:
        chosen = rf_model if model == "rf" else xgb_model
        cols = RF_COLS if model == "rf" else XGB_COLS
        X = align_to_schema(evt.payload, cols)

        prob = float(chosen.predict_proba(X)[:, 1][0]) if hasattr(chosen, "predict_proba") else None
        pred = int(chosen.predict(X)[0])

        rec = {
            "ts": ts, "model": model, "prediction": pred, "probability": prob,
            "received_keys": list(evt.payload.keys())
        }
        log_prediction(rec)
        return {**rec, "missing_columns_filled": [c for c in cols if c not in evt.payload]}
    except Exception as e:
        return {"error": str(e), "payload_received": evt.payload, "ts": ts}

@app.post("/explain")
def explain(evt: TrafficEvent, model: str = Query("rf", enum=["rf", "xgb"]), top_k: int = 10):
    """
    SHAP-based local explanation for tree models.
    Works by transforming with the pipeline preprocessor, then explaining the tree model.
    """
    ts = time.time()
    try:
        chosen = rf_model if model == "rf" else xgb_model
        cols = RF_COLS if model == "rf" else XGB_COLS
        pre = chosen.named_steps["pre"]
        clf = chosen.named_steps["clf"]

        # align raw -> transform -> dense
        X_raw = align_to_schema(evt.payload, cols)
        X_tx = pre.transform(X_raw)
        X_tx = X_tx.toarray() if hasattr(X_tx, "toarray") else X_tx
        feat_names = list(pre.get_feature_names_out())

        # TreeExplainer for RF/XGB (binary): pick class 1 shap values
        explainer = shap.TreeExplainer(clf)
        shap_vals = explainer.shap_values(X_tx)
        if isinstance(shap_vals, list):  # XGB returns [class0, class1]
            shap_row = shap_vals[1][0]
        else:
            shap_row = shap_vals[0]

        # top-k features by |contribution|
        pairs = sorted(
            [{"feature": f, "shap": float(v), "abs_shap": float(abs(v))} for f, v in zip(feat_names, shap_row)],
            key=lambda d: d["abs_shap"],
            reverse=True
        )[:max(1, top_k)]

        # prediction & prob for context
        prob = float(chosen.predict_proba(X_raw)[:, 1][0]) if hasattr(chosen, "predict_proba") else None
        pred = int(chosen.predict(X_raw)[0])

        return {
            "ts": ts, "model": model, "prediction": pred, "probability": prob,
            "top_contributors": pairs, "feature_count": len(feat_names)
        }
    except Exception as e:
        return {"error": str(e), "payload_received": evt.payload, "ts": ts}


In [ ]:
import nest_asyncio, uvicorn
from pyngrok import ngrok

nest_asyncio.apply()
public_url = ngrok.connect(8000, "http")
print("Public URL:", public_url)
uvicorn.run(app, host="0.0.0.0", port=8000)


Public URL: NgrokTunnel: "https://4de7d614318f.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [151]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     180.233.124.104:0 - "GET /doc HTTP/1.1" 404 Not Found
INFO:     180.233.124.104:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     180.233.124.104:0 - "GET /doc HTTP/1.1" 404 Not Found
INFO:     180.233.124.104:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     180.233.124.104:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     180.233.124.104:0 - "POST /predict?model=rf HTTP/1.1" 200 OK
INFO:     180.233.124.104:0 - "POST /predict?model=rf HTTP/1.1" 200 OK
